## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---
## First, I'll compute the camera calibration using chessboard images

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib inline

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal\calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    
    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

    # If found, add object points, image points
    if ret == True:
        print(fname)
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
#        cv2.imshow('img',img)
        cv2.imwrite("output_images\calibration_corners.jpg", img)
#        cv2.waitKey(500)

cv2.destroyAllWindows()

objpointsArr = np.array(objpoints)
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints,imgpoints, gray.shape[::-1], None, None)

camera_cal\calibration10.jpg
camera_cal\calibration11.jpg
camera_cal\calibration12.jpg
camera_cal\calibration13.jpg
camera_cal\calibration14.jpg
camera_cal\calibration15.jpg
camera_cal\calibration16.jpg
camera_cal\calibration17.jpg
camera_cal\calibration18.jpg
camera_cal\calibration19.jpg
camera_cal\calibration2.jpg
camera_cal\calibration20.jpg
camera_cal\calibration3.jpg
camera_cal\calibration6.jpg
camera_cal\calibration7.jpg
camera_cal\calibration8.jpg
camera_cal\calibration9.jpg


## Function to undistort images

In [2]:
def cal_undistort(img):
    # Use cv2.calibrateCamera() and cv2.undistort()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist



In [3]:
def detect_edges(img, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HSV color space and separate the V channel
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hsv[:,:,1]
    s_channel = hsv[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 255
    
    # Threshold color channel
    s_binary = np.zeros_like(scaled_sobel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 255
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.

    color_binary = np.zeros_like(scaled_sobel).astype(np.uint8)
    color_binary[(s_binary > 0) | (sxbinary > 0)] = 255
    
    return color_binary

In [11]:
import matplotlib.pyplot as plt

class LineFitting:
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_leftx = [] 
        self.recent_lefty = [] 
        self.recent_rightx = [] 
        self.recent_righty = [] 
        #polynomial coefficients averaged over the last n iterations
        self.best_left_fit = None  
        self.best_right_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_left_fit = [np.array([False])]  
        self.current_right_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.left_radius_of_curvature = None 
        self.right_radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        
    def find_line_fit(self, binary_warped, Minv):
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        self.recent_leftx = nonzerox[left_lane_inds]
        self.recent_lefty = nonzeroy[left_lane_inds] 
        self.recent_rightx = nonzerox[right_lane_inds]
        self.recent_righty = nonzeroy[right_lane_inds] 
        leftx = self.recent_leftx
        lefty = self.recent_lefty
        rightx = self.recent_rightx
        righty = self.recent_righty
        
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        if (self.detected):
            left_diff_prev = abs(left_fit - self.current_left_fit)
            right_diff_prev = abs(right_fit - self.current_right_fit)
            diff_fit = abs(left_fit - right_fit)
            # are lanes parallel ?
            if (diff_fit[0] > 0.005 or diff_fit[1] > 0.3):
                self.detected = False
                left_fit = self.current_left_fit
                right_fit = self.current_right_fit
                
            # do lanes diverge too much from previous frame?
            if (left_diff_prev[0] > 0.001 or right_diff_prev[0] > 0.001 or
               left_diff_prev[1] > 0.5 or right_diff_prev[1] > 0.5 or
               left_diff_prev[2] > 500 or right_diff_prev[2] > 500):
                print(left_diff_prev, right_diff_prev)
                self.detected = False
                left_fit = self.current_left_fit
                right_fit = self.current_right_fit
                
        left_fit_avg = []
        right_fit_avg = []
        if (self.detected ):
            left_fit_avg = (left_fit + self.current_left_fit) / float(2)
            right_fit_avg = (right_fit + self.current_right_fit) / float(2)
        else:
            left_fit_avg = left_fit
            right_fit_avg = right_fit
        
        self.current_left_fit = left_fit
        self.current_right_fit = right_fit
        self.best_left_fit = left_fit_avg
        self.best_right_fit = right_fit_avg
        
        left_fit = self.best_left_fit
        right_fit = self.best_right_fit
        left_eval = np.max(self.recent_lefty)
        right_eval = np.max(self.recent_righty)
        left_curverad = ((1 + (2*left_fit[0]*left_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
        right_curverad = ((1 + (2*right_fit[0]*right_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])

        # Define conversions in x and y from pixels space to meters
        ym_per_pix = 30/720 # meters per pixel in y dimension
        xm_per_pix = 3.7/700 # meters per pixel in x dimension

        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
        # Calculate the new radii of curvature
        self.left_radius_of_curvature = ((1 + (2*left_fit_cr[0]*left_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        self.right_radius_of_curvature = ((1 + (2*right_fit_cr[0]*right_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
      
        # measure offset from center
        offset = 100
        y_bottom = binary_warped.shape[0] - offset
        x_left = left_fit[0]*y_bottom*y_bottom + left_fit[1]*y_bottom + left_fit[2]
        x_right = right_fit[0]*y_bottom*y_bottom + right_fit[1]*y_bottom + right_fit[2]
        x_center = int((x_left + x_right) / 2)
        x_image_center = binary_warped.shape[1] / 2
        self.line_base_pos = abs(x_center - x_image_center)*xm_per_pix
      
        warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = cv2.warpPerspective(color_warp, Minv, (binary_warped.shape[1], binary_warped.shape[0]))
        self.detected = True
        """
        plt.figure()
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
        plt.imshow(out_img)
        plt.plot(left_fitx, ploty, color='green', linewidth=3)
        plt.plot(right_fitx, ploty, color='green', linewidth=3)
        plt.xlim(0, 1280)
        plt.ylim(720, 0)
        """
        return newwarp

In [12]:
def pipeline(img, lineFit):
    undistorted = cal_undistort(img)
    #cv2.imwrite("output_images/undistorted.jpg", undistorted)
    edges = detect_edges(undistorted)
    #cv2.imwrite("output_images/binary.jpg", edges)
    edges = np.array(edges)
    
    #edges = undistorted
    offset = 180
    img_size = (edges.shape[1], edges.shape[0])
    src = np.float32([[560,475], [724, 475], [1028,670], [270,670]])
    dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                     [img_size[0]-offset, img_size[1]-offset], 
                     [offset, img_size[1]-offset]])

    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    # Warp the image using OpenCV warpPerspective()
    
    warped = cv2.warpPerspective(edges, M, img_size)
    #cv2.imwrite("output_images/warped.jpg", warped)
    new_warped = lineFit.find_line_fit(warped, Minv)
    result = cv2.addWeighted(undistorted[200:,:], 1, new_warped[200:,:], 0.3, 0)
    radius_curve = max(lineFit.left_radius_of_curvature, lineFit.left_radius_of_curvature)
    radius_curve_str = "Radius of Curvature: " + str(radius_curve) + " m"
    center_offset_str = "Offset from center: " + str("{:1.4f}".format(lineFit.line_base_pos)) + " m"
    cv2.putText(result, radius_curve_str, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, 255)
    cv2.putText(result, center_offset_str, (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1, 255)
    return result

In [6]:
test_images = glob.glob("test_images\*.jpg")
count = 0
for filename in test_images:
    print (filename)
    img = cv2.imread(filename)
    lineFit = LineFitting()
    warped = pipeline(img, lineFit)
    output_name = "warped" + str(count) + ".jpg"
    count = count + 1
    cv2.imwrite("output_images\\" + output_name, warped)

test_images\straight_lines1.jpg
test_images\straight_lines2.jpg
test_images\test1.jpg
test_images\test2.jpg
test_images\test3.jpg
test_images\test4.jpg
test_images\test5.jpg
test_images\test6.jpg


In [7]:
# Import everything needed to edit/save/watch video clips
import imageio
imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [15]:
lineFit = LineFitting()
count  = 0
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    global lineFit
    global count
    count = count + 1
   
    warped = pipeline(image, lineFit)
    return warped

In [16]:
white_output = 'project_video_result.mp4'
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video project_video_result.mp4
[MoviePy] Writing video project_video_result.mp4


 45%|████▌     | 569/1261 [01:28<01:32,  7.49it/s]

[  6.42921496e-04   5.96198842e-01   1.35493875e+02] [  3.18071988e-04   1.90018466e-01   1.30602788e+01]


 46%|████▌     | 576/1261 [01:29<01:34,  7.26it/s]

[  5.34467939e-04   5.02060313e-01   1.20047616e+02] [  2.44069326e-04   9.04348255e-02   4.92728236e+00]


...

[  2.07914334e-06   1.17342291e-01   5.95426508e+01] [  1.08664723e-02   1.13882340e+01   3.37068740e+03]


 97%|█████████▋| 1228/1261 [03:10<00:05,  6.49it/s]

[  1.34000831e-04   1.85125585e-01   5.58731169e+01] [  5.48682956e-04   5.12042643e-01   1.02323723e+02]


100%|█████████▉| 1260/1261 [03:15<00:00,  6.55it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_result.mp4 

Wall time: 3min 17s
